In [29]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/company-bankruptcy-prediction/data.csv


In [30]:
from keras.layers import BatchNormalization, Dense, Dropout
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn import metrics   
from sklearn.model_selection import KFold

In [31]:
df = pd.read_csv('../input/company-bankruptcy-prediction/data.csv')
Y = df['Bankrupt?']
X = df.drop(['Bankrupt?'], axis=1)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state = 42)

In [32]:
inputs = np.concatenate((X_train, X_test), axis=0)
targets = np.concatenate((Y_train, Y_test), axis=0)

acc_per_fold = []
loss_per_fold = []

fold_no = 1
num_folds = 10

kfold = KFold(n_splits=num_folds, shuffle=True)

# K-fold Cross Validation model evaluation
for train, test in kfold.split(inputs, targets):

    # Cool model stuff (Did some off notebook tweaking to get the best model!)
    model = Sequential()

    model.add(Dense(64, activation='relu', input_shape=(95,)))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))

    model.add(Dense(64, activation='relu')) 
    model.add(BatchNormalization())
    model.add(Dropout(0.25))

    model.add(Dense(1, activation='sigmoid'))
    

    checkpoint = ModelCheckpoint('BankruptcyModel.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)
    earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=25,
                          verbose=1,
                          restore_best_weights=True
                          )
    reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=3,
                              verbose=1,
                              min_delta=0.0001)
    # This will help with accuracy
    callbacks = [earlystop,checkpoint,reduce_lr]
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Asthetics
    print('------------------------------------------------------------------------')
    print(f'Training Fold {fold_no} / {num_folds}')

    history = model.fit(inputs[train], targets[train],
              epochs=25,
              callbacks = callbacks,
              verbose=1)

    # Generalization metrics
    scores = model.evaluate(inputs[test], targets[test], verbose=1)
    
    print(f'Score for Fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    fold_no = fold_no + 1

------------------------------------------------------------------------
Training Fold 1 / 10
Epoch 1/25
192/192 [==============================] - 1s 2ms/step - loss: 0.6495 - accuracy: 0.6789
Epoch 2/25
192/192 [==============================] - 0s 2ms/step - loss: 0.2251 - accuracy: 0.9541
Epoch 3/25
192/192 [==============================] - 0s 2ms/step - loss: 0.1597 - accuracy: 0.9648
Epoch 4/25
192/192 [==============================] - 0s 2ms/step - loss: 0.1612 - accuracy: 0.9645
Epoch 5/25
192/192 [==============================] - 0s 2ms/step - loss: 0.1550 - accuracy: 0.9637
Epoch 6/25
192/192 [==============================] - 0s 2ms/step - loss: 0.1436 - accuracy: 0.9640
Epoch 7/25
192/192 [==============================] - 0s 2ms/step - loss: 0.1477 - accuracy: 0.9659
Epoch 8/25
192/192 [==============================] - 0s 2ms/step - loss: 0.1346 - accuracy: 0.9681
Epoch 9/25
192/192 [==============================] - 0s 2ms/step - loss: 0.1229 - accuracy: 0.9727
Epoch 

22/22 [==============================] - 0s 966us/step - loss: 0.1136 - accuracy: 0.9707
Score for Fold 3: loss of 0.11359000205993652; accuracy of 97.06745147705078%
------------------------------------------------------------------------
Training Fold 4 / 10
Epoch 1/25
192/192 [==============================] - 1s 2ms/step - loss: 0.6542 - accuracy: 0.6537
Epoch 2/25
192/192 [==============================] - 0s 2ms/step - loss: 0.2354 - accuracy: 0.9491
Epoch 3/25
192/192 [==============================] - 0s 2ms/step - loss: 0.1538 - accuracy: 0.9661
Epoch 4/25
192/192 [==============================] - 0s 2ms/step - loss: 0.1531 - accuracy: 0.9653
Epoch 5/25
192/192 [==============================] - 0s 2ms/step - loss: 0.1453 - accuracy: 0.9673
Epoch 6/25
192/192 [==============================] - 0s 2ms/step - loss: 0.1429 - accuracy: 0.9657
Epoch 7/25
192/192 [==============================] - 0s 2ms/step - loss: 0.1502 - accuracy: 0.9652
Epoch 8/25
192/192 [===================

192/192 [==============================] - 0s 2ms/step - loss: 0.1162 - accuracy: 0.9697
Epoch 25/25
22/22 [==============================] - 0s 1ms/step - loss: 0.1404 - accuracy: 0.9633
Score for Fold 6: loss of 0.14040081202983856; accuracy of 96.334308385849%
------------------------------------------------------------------------
Training Fold 7 / 10
Epoch 1/25
192/192 [==============================] - 1s 2ms/step - loss: 0.6587 - accuracy: 0.6685
Epoch 2/25
192/192 [==============================] - 0s 2ms/step - loss: 0.2224 - accuracy: 0.9474
Epoch 3/25
192/192 [==============================] - 0s 2ms/step - loss: 0.1610 - accuracy: 0.9635
Epoch 4/25
192/192 [==============================] - 0s 2ms/step - loss: 0.1636 - accuracy: 0.9623
Epoch 5/25
192/192 [==============================] - 0s 2ms/step - loss: 0.1259 - accuracy: 0.9701
Epoch 6/25
192/192 [==============================] - 0s 2ms/step - loss: 0.1400 - accuracy: 0.9660
Epoch 7/25
192/192 [======================

192/192 [==============================] - 0s 2ms/step - loss: 0.1171 - accuracy: 0.9705
Epoch 23/25
192/192 [==============================] - 0s 2ms/step - loss: 0.1271 - accuracy: 0.9696
Epoch 24/25
192/192 [==============================] - 0s 2ms/step - loss: 0.1243 - accuracy: 0.9682
Epoch 25/25
22/22 [==============================] - 0s 1ms/step - loss: 0.1386 - accuracy: 0.9663
Score for Fold 9: loss of 0.13862299919128418; accuracy of 96.62756323814392%
------------------------------------------------------------------------
Training Fold 10 / 10
Epoch 1/25
192/192 [==============================] - 1s 2ms/step - loss: 0.6680 - accuracy: 0.6805
Epoch 2/25
192/192 [==============================] - 0s 2ms/step - loss: 0.2341 - accuracy: 0.9470
Epoch 3/25
192/192 [==============================] - 0s 2ms/step - loss: 0.1549 - accuracy: 0.9664
Epoch 4/25
192/192 [==============================] - 0s 2ms/step - loss: 0.1366 - accuracy: 0.9677
Epoch 5/25
192/192 [=================

In [33]:
# Asthetics again!
print('------------------------------------------------------------------------')
print('Score per Fold')
print('------------------------------------------------------------------------')
# Print for how many ever folds
for i in range(0, len(acc_per_fold)):
    #print('------------------------------------------------------------------------')
    print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
# This'll give us a good idea of the leeway for our accuracy
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')
lowAvg = (np.mean(acc_per_fold)) - (np.std(acc_per_fold))
highAvg = (np.mean(acc_per_fold)) + (np.std(acc_per_fold))
lowAvg = (round(lowAvg, 2))
highAvg = (round(highAvg, 2))
print(f'> Expect anywhere from: {lowAvg}% - {highAvg}% model accuracy')

------------------------------------------------------------------------
Score per Fold
------------------------------------------------------------------------
> Fold 1 - Loss: 0.11468835920095444 - Accuracy: 97.3607063293457%
> Fold 2 - Loss: 0.14338241517543793 - Accuracy: 96.77419066429138%
> Fold 3 - Loss: 0.11359000205993652 - Accuracy: 97.06745147705078%
> Fold 4 - Loss: 0.13220366835594177 - Accuracy: 96.62756323814392%
> Fold 5 - Loss: 0.14365985989570618 - Accuracy: 96.18768095970154%
> Fold 6 - Loss: 0.14040081202983856 - Accuracy: 96.334308385849%
> Fold 7 - Loss: 0.14629295468330383 - Accuracy: 96.48093581199646%
> Fold 8 - Loss: 0.1412932276725769 - Accuracy: 96.92082405090332%
> Fold 9 - Loss: 0.13862299919128418 - Accuracy: 96.62756323814392%
> Fold 10 - Loss: 0.1315024346113205 - Accuracy: 96.91630005836487%
------------------------------------------------------------------------
Average scores for all folds:
> Accuracy: 96.72975242137909 (+- 0.3344307178376142)
> Loss

In [34]:
# Nice! This avg acc tell's us our general for the entire dataset so that we aren't just getting 99% in one split & 20% in the other. Fun Stuff :)\

In [ ]:
inputs = np.concatenate((X_train, X_test), axis=0)
targets = np.concatenate((Y_train, Y_test), axis=0)

acc_per_fold = []
loss_per_fold = []

fold_no = 1
num_folds = 25

kfold = KFold(n_splits=num_folds, shuffle=True)

# K-fold Cross Validation model evaluation
for train, test in kfold.split(inputs, targets):

    # Cool model stuff (Did some off notebook tweaking to get the best model!)
    model = Sequential()

    model.add(Dense(64, activation='relu', input_shape=(95,)))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))

    model.add(Dense(64, activation='relu')) 
    model.add(BatchNormalization())
    model.add(Dropout(0.25))

    model.add(Dense(1, activation='sigmoid'))
    

    checkpoint = ModelCheckpoint('BankruptcyModel.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)
    earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=25,
                          verbose=1,
                          restore_best_weights=True
                          )
    reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=3,
                              verbose=1,
                              min_delta=0.0001)
    # This will help with accuracy
    callbacks = [earlystop,checkpoint,reduce_lr]
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Asthetics
    print('------------------------------------------------------------------------')
    print(f'Training Fold {fold_no} / {num_folds}')

    history = model.fit(inputs[train], targets[train],
              epochs=25,
              callbacks = callbacks,
              verbose=1)

    # Generalization metrics
    scores = model.evaluate(inputs[test], targets[test], verbose=1)
    
    print(f'Score for Fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    fold_no = fold_no + 1

------------------------------------------------------------------------
Training Fold 1 / 25
Epoch 1/25
205/205 [==============================] - 1s 2ms/step - loss: 0.6441 - accuracy: 0.6721
Epoch 2/25
205/205 [==============================] - 0s 2ms/step - loss: 0.2165 - accuracy: 0.9521
Epoch 3/25
205/205 [==============================] - 0s 2ms/step - loss: 0.1662 - accuracy: 0.9632
Epoch 4/25
205/205 [==============================] - 0s 2ms/step - loss: 0.1459 - accuracy: 0.9674
Epoch 5/25
205/205 [==============================] - 0s 2ms/step - loss: 0.1314 - accuracy: 0.9692
Epoch 6/25
205/205 [==============================] - 0s 2ms/step - loss: 0.1339 - accuracy: 0.9682
Epoch 7/25
205/205 [==============================] - 0s 2ms/step - loss: 0.1504 - accuracy: 0.9636
Epoch 8/25
205/205 [==============================] - 0s 2ms/step - loss: 0.1298 - accuracy: 0.9690
Epoch 9/25
205/205 [==============================] - 0s 2ms/step - loss: 0.1452 - accuracy: 0.9632
Epoch 

9/9 [==============================] - 0s 1ms/step - loss: 0.1709 - accuracy: 0.9560
Score for Fold 3: loss of 0.17094926536083221; accuracy of 95.60439586639404%
------------------------------------------------------------------------
Training Fold 4 / 25
Epoch 1/25
205/205 [==============================] - 1s 2ms/step - loss: 0.6501 - accuracy: 0.6610
Epoch 2/25
205/205 [==============================] - 0s 2ms/step - loss: 0.2141 - accuracy: 0.9579
Epoch 3/25
205/205 [==============================] - 0s 2ms/step - loss: 0.1575 - accuracy: 0.9660
Epoch 4/25
205/205 [==============================] - 0s 2ms/step - loss: 0.1499 - accuracy: 0.9657
Epoch 5/25
205/205 [==============================] - 0s 2ms/step - loss: 0.1557 - accuracy: 0.9640
Epoch 6/25
205/205 [==============================] - 0s 2ms/step - loss: 0.1356 - accuracy: 0.9697
Epoch 7/25
205/205 [==============================] - 0s 2ms/step - loss: 0.1392 - accuracy: 0.9661
Epoch 8/25
205/205 [=======================

205/205 [==============================] - 0s 2ms/step - loss: 0.1272 - accuracy: 0.9688
Epoch 25/25
9/9 [==============================] - 0s 2ms/step - loss: 0.0727 - accuracy: 0.9817
Score for Fold 6: loss of 0.07267742604017258; accuracy of 98.16849827766418%
------------------------------------------------------------------------
Training Fold 7 / 25
Epoch 1/25
205/205 [==============================] - 1s 2ms/step - loss: 0.6410 - accuracy: 0.6793


In [ ]:
# Asthetics again!
print('------------------------------------------------------------------------')
print('Score per Fold')
print('------------------------------------------------------------------------')
# Print for how many ever folds
for i in range(0, len(acc_per_fold)):
    #print('------------------------------------------------------------------------')
    print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
# This'll give us a good idea of the leeway for our accuracy
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')
lowAvg = (np.mean(acc_per_fold)) - (np.std(acc_per_fold))
highAvg = (np.mean(acc_per_fold)) + (np.std(acc_per_fold))
lowAvg = (round(lowAvg, 2))
highAvg = (round(highAvg, 2))
print(f'> Expect anywhere from: {lowAvg}% - {highAvg}% model accuracy')

In [ ]:
# And Voila! Increased fold number: Got better generalization, similar average as last time, with a wider range of accuracy :)\